In [25]:
import pandas as pd
import matplotlib
import requests

GitHubUserURLBase = "https://api.github.com/users/"
GitHubPublicEvents = "/events/public"

class GitHubUser:
    def __init__(self, username):
        self.username = username
    def get_github_user(self):
        UserInfo = requests.get(GitHubUserURLBase + self.username)
        UserInfoUsable = UserInfo.json()
        return(UserInfoUsable)
    def verify_github_user(self):
        UserInfo = self.get_github_user()
        if 'login' in UserInfo:
            return("User_Exists")
        else:
            raise ValueError("Not a valid GitHub user")
    def get_github_user_activity(self):
        UserActivity = requests.get(GitHubUserURLBase + self.username + GitHubPublicEvents ) # + "?page=" + PageStr
        UserActivityPage = UserActivity.json()
        return(UserActivityPage)

# detaer = GitHubUser('detaer')
# detaeruser = detaer.get_github_user_activity()
# print(detaeruser)

##Create an object to convert the github user events into a pandas dataframe and format it for two different uses
## use 1 hourly activity, use 2 day of the week activity


##create a method to plot the daily activity of a user week over week day over day then display it


##create a method to plot the hourly activity of a user day over day then display it

##create main - this will need to take user input of the git hub username
##TODO take the input as a string (can't figure this out right now)
def main():
    UserName = input("Enter the GitHub username of the person you want to creep on:")
    User = GitHubUser(UserName)
    ##do a try except to throw an error if the user isn't real, if the user is real, grab their public activity
    try:
        TestUser = User.verify_github_user()
        print(TestUser)
    except:
        print("User is not a valid github user!")
    
    
        

if __name__ == '__main__':
    main()


Enter the GitHub username of the person you want to creep on:detaerer
User is not a valid github user!


In [18]:
import requests
GitHubUserURLBase = "https://api.github.com/users/"
GitHubPublicEvents = "/events/public"

class GitHubUser:
    def __init__(self, username):
        self.username = username
    def get_github_user(self):
        print(requests.get(GitHubUserURLBase + self.username))
        UserInfoUsable = UserInfo.json()
        return(UserInfoUsable)
    def verify_github_user(self):
        UserInfo = self.get_github_user()
        if 'login' in UserInfo:
            return("User_Exists")
        else:
            return("Something_Broke")
    def get_github_user_activity(self):
        UserActivity = requests.get(GitHubUserURLBase + self.username + GitHubPublicEvents ) # + "?page=" + PageStr
        UserActivityPage = UserActivity.json()
        return(UserActivityPage)

print(GitHubUser.get_github_user(detaer))
# detaer = GitHubUser('detaer')
# detaeruser = detaer.get_github_user_activity()
# print(detaeruser)

<Response [200]>


NameError: name 'UserInfo' is not defined

In [38]:
##create main - this will need to take user input of the git hub username
##TODO take the input as a string (can't figure this out right now)
def main():
    UserName = input("Enter the GitHub username of the person you want to creep on:")
##do a try except to throw an error if the user isn't real, if the user is real, grab their public activity
    try:
        GetGitHubUserinfo(UserName) = User_Exists
    except:
        print("Get better at stalking, that person isn't even real!")


if __name__ == '__main__':
    main()

SyntaxError: can't assign to function call (<ipython-input-38-b04454a16708>, line 7)

In [39]:
list_of_users = ['detaer', 'cpleasants']
for user in list_of_users:
    this_user = GitHubUser(user)
    print(this_user.verify_github_user())

NameError: name 'GitHubUser' is not defined

In [40]:
detaer = GitHubUser('detaer')
cpleasants = GitHubUser('cpleasants')

NameError: name 'GitHubUser' is not defined

In [41]:
ids = []
types = []
times = []
for activity in detaer.get_github_user_activity():
    ids.append(activity['id'])
    types.append(activity['type'])
    times.append(activity['created_at'])
    
df = pd.DataFrame({'id' : ids,
         'type' : types,
         'created_at' : times})

NameError: name 'detaer' is not defined

In [42]:
`df['created_at'] = pd.to_datetime(df['created_at'])`
`df['hour_of_day'] = df.created_at.dt.hour`

SyntaxError: invalid syntax (<ipython-input-42-0db7170d10ef>, line 1)

In [43]:
from datetime import timedelta
df['created_at'] + timedelta(hours = 8)

NameError: name 'df' is not defined

In [ ]:
%matplotlib inline